In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np



In [2]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
189,"Calling all D-sciples! Grab your friends, hit ...",positive
576,Whilst I have loved haunted house movies such ...,positive
261,This is certainly one of my all time fav episo...,positive
689,"This movie surprised me. Some things were ""cli...",positive
17,WE FAW DOWN <br /><br />Aspect ratio: 1.33:1<b...,negative


In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
21,What on earth has become of our dear Ramu? Is ...,negative
521,"LIGHTS OF NEW YORK was the first ""all-taking"" ...",positive
485,"I don't expect a lot from ghost stories, but I...",negative
939,There are plenty of comments already posted sa...,negative
858,Being an unrelenting non-stop over-the-top exp...,negative


In [4]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [5]:
df = normalize_text(df)
df.head()

,review,sentiment
21,earth become dear ramu man made sarkar satya c...,negative
521,light new york first all taking feature film c...,positive
485,expect lot ghost story expect story make bit s...,negative
939,plenty comment already posted saying exactly f...,negative
858,unrelenting non stop over the top explosive me...,negative


In [6]:
df['sentiment'].value_counts()

sentiment
positive    252
negative    248
Name: count, dtype: int64

In [7]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [8]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
21,earth become dear ramu man made sarkar satya c...,0
521,light new york first all taking feature film c...,1
485,expect lot ghost story expect story make bit s...,0
939,plenty comment already posted saying exactly f...,0
858,unrelenting non stop over the top explosive me...,0


In [9]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [14]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/Jittub45/Capstone-.mlflow')
dagshub.init(repo_owner='Jittub45', repo_name='Capstone-Project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


2025-04-29 00:57:57,559 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/Jittub45/Capstone-Project "HTTP/1.1 200 OK"


Initialized MLflow to track repo "Jittub45/Capstone-Project"

2025-04-29 00:57:57,571 - INFO - Initialized MLflow to track repo "Jittub45/Capstone-Project"


Repository Jittub45/Capstone-Project initialized!

2025-04-29 00:57:57,580 - INFO - Repository Jittub45/Capstone-Project initialized!


<Experiment: artifact_location='mlflow-artifacts:/2cb8161c13214c5bbf24f9e207fb6487', creation_time=1745868231800, experiment_id='0', last_update_time=1745868231800, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [17]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.2)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-04-29 00:57:59,442 - INFO - Starting MLflow run...


2025-04-29 00:57:59,913 - INFO - Logging preprocessing parameters...
2025-04-29 00:58:01,245 - INFO - Initializing Logistic Regression model...
2025-04-29 00:58:01,247 - INFO - Fitting the model...
2025-04-29 00:58:01,282 - INFO - Model training complete.
2025-04-29 00:58:01,282 - INFO - Logging model parameters...
2025-04-29 00:58:01,754 - INFO - Making predictions...
2025-04-29 00:58:01,758 - INFO - Calculating evaluation metrics...
2025-04-29 00:58:01,781 - INFO - Logging evaluation metrics...
2025-04-29 00:58:03,702 - INFO - Saving and logging the model...
2025/04/29 00:58:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-04-29 00:58:13,838 - INFO - Model training and logging completed in 13.93 seconds.
2025-04-29 00:58:13,839 - INFO - Accuracy: 0.65
2025-04-29 00:58:13,840 - INFO - Precision: 0.66
2025-04-29 00:58:13,841 - INFO - Recall: 0.647058823

🏃 View run thoughtful-whale-644 at: https://dagshub.com/Jittub45/Capstone-.mlflow/#/experiments/0/runs/fa29c953d3a04d7b91d37d127f11a88b
🧪 View experiment at: https://dagshub.com/Jittub45/Capstone-.mlflow/#/experiments/0
